<a href="https://colab.research.google.com/github/mksong17/storytelling-with-data/blob/master/data-stories/Police_killings/PoliceViolence.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#number crunching
import numpy as np
import pandas as pd

#data import
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials
import pandas_datareader as pdr

#data visualization
import plotly
import plotly.express as px
from plotly import graph_objects as go
import seaborn as sn

/usr/local/lib/python3.6/dist-packages/pandas_datareader/compat/__init__.py:7: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  from pandas.util.testing import assert_frame_equal


In [0]:
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)

# Project team
Arvin Kumaran, Elliott Tang, MK Song
Background and Overview

# Overview
It is a widely held belief that police are meant to protect. In 2015, protests erupted across the country after police wrongfully killed Michael Brown in Ferguson, Missouri and Eric Garner in New York City. These demonstrations led to the rise of the Black Lives Matter movement. One would think that after years of protest, the American police force would have adapted their practices to promote values of racial awareness and egalitarianism. However, recent tragic events such as the death of African Americans like George Floyd, Ahmaud Arbery, and Breona Taylor at the hands of police officers have once again thrust the spotlight back onto police methods of violence and brutality. We must ask the question: are our police just arbiters of safety and law, or are they “criminals of permission” who contribute to racial and social fears in our country? In our analysis, we decided to focus on this very question by analyzing data regarding police killings in tandem with factors such as race and geography.

#Approach

 We found a comprehensive dataset containing 2013-2019 police killings by state with a comprehensive set of numbers revolving around race disparities, populations, and geographies. There were four separate components of this dataset: (1)details per case of 2013-2019 police killings, (2) the demographics of these 2013-2019 police killings, (3) killings by state, and finally (4) police killings of black males. We began our analysis by looking at the difference in rates of murders for black people versus rates of murders for all people. Next, we analyzed the disparities of these two rates by state and then we compared the police violence rate to the violent crime rate per state. We looked at the racial composition of killings per million people. Finally, we looked at whether a police officer was charged after a homicidal encounter. 
 


#Quick Summary

We found out that black people are killed at a much higher rate than all other people in EVERY SINGLE STATE. Interestingly, the disparity between the rate at which black citizens were killed compared to all other citizens was highest in strongly Republican states like Alaska, Utah, and Oklahoma. We also discovered that the police homicide rate had no correlation at all with the crime rate in the city. This was intriguing, as we would expect to see more police killings in areas with higher violent crime rates. This suggests that cops are more violent in areas where such violent methods aren’t even truly warranted. We decided to explore the racial composition of police killings per million people in big cities. We found that black people were killed 2x as more than Latino and 3x as more than White people. They were 1.13x more likely to be killed than Latino and 1.3x more likely than White People when they are unarmed. This led us to the conclusion that black people were most likely to be killed by police. This may reflect a racial bias inherent within police forces around the country, but it may also be indicative of a bias broadly inherent in our society. Finally, we discovered that 99% of police officers who have committed murder were not charged after the killing. 




# Data

In [0]:
policekillingsvictims = pd.read_excel('https://github.com/mksong17/storytelling-with-data/raw/master/data-stories/Police_killings/Police_killing.xlsx', sheet_name = 0)
killingsbypd = pd.read_excel('https://github.com/mksong17/storytelling-with-data/raw/master/data-stories/Police_killings/Police_killing.xlsx', sheet_name = 1)
killingsbystate = pd.read_excel('https://github.com/mksong17/storytelling-with-data/raw/master/data-stories/Police_killings/Police_killing.xlsx', sheet_name = 2)
blackmenkillings = pd.read_excel('https://github.com/mksong17/storytelling-with-data/raw/master/data-stories/Police_killings/Police_killing.xlsx', sheet_name = 3)


In [3]:
bp = go.Bar(
            name='Rate (Black People)',
            x=killingsbystate['State'],
            y=killingsbystate['Rate (Black People)'],
            offsetgroup=0)
ap = go.Bar(name="Rate (All People)",
            x=killingsbystate['State'],
            y=killingsbystate['Rate (All People)'],
            offsetgroup=1)
data = [bp, ap]
plotly.offline.iplot({
    "data": data,
    "layout": go.Layout(barmode='group')
})

In [0]:
px.bar(killingsbystate, y = 'Disparity in Rate', x = 'State')

In [0]:
##numpy argsort function

##create a new df with columns, then sort, then graph 
##pandas.iloc
#ratesdf = killingsbypd 
#ratesdf['Violent Crime Rate'] = ratesdf['Violent Crime Rate'].to_frame().sort_values(by = 'Violent Crime Rate', ascending = True)
#ratesdf['Avg Annual Police Homicide Rate'] = ratesdf['Avg Annual Police Homicide Rate'].to_frame().sort_values(by = 'Avg Annual Police Homicide Rate', ascending = True)
#ratesdf

rates = go.Figure()





rates.add_trace(go.Scatter(x=killingsbypd['City'], y=killingsbypd['Avg Annual Police Homicide Rate'],
                   mode='markers',
                  name='Police Homicide Rate'))
rates.add_trace(go.Scatter(x=killingsbypd['City'], y=killingsbypd['Violent Crime Rate'],
                  mode='markers',
                name='Violent Crime Rate'))


rates.show()



#ratesdf['Violent Crime Rate']

In [0]:
policekillingsvictims[["Victim's race", "Unarmed"]]
unarmed = policekillingsvictims.loc[policekillingsvictims["Unarmed"]== 'Unarmed']
unarmed = unarmed.rename(columns = {"Victim's race":'Race'})
unarmed = unarmed.groupby('Race').size()
unarmed = unarmed.to_frame()
totalkilled = policekillingsvictims[["Victim's race", "Unarmed"]]
totalkilled = totalkilled.rename(columns = {"Victim's race":'Race'})
totalkilled = totalkilled.groupby('Race').size()
totalkilled = totalkilled.to_frame()
comp_unarmed = (unarmed/totalkilled)*100
comp_unarmed = comp_unarmed.assign(Race=comp_unarmed.index.get_level_values('Race'))
comp_unarmed.columns = ['Ratio', 'Race']
wbl = comp_unarmed.loc[(comp_unarmed['Race'] == 'Black')|(comp_unarmed['Race'] == 'Hispanic')|(comp_unarmed['Race'] == 'White')]
wbl

,Ratio,Race
Race,,
Black,17.275064,Black
Hispanic,15.044910,Hispanic
White,13.376739,White


In [0]:
px.bar(wbl, x = 'Race', y = 'Ratio', opacity = 0.70, title = 'The Ratio of People Killed by Police when Unarmed', color = 'Race')

In [0]:
 bigcities = killingsbypd.iloc[[100]]
bigcities_killed = bigcities[["Black People Killed by Police (1/1/2013-12/31/2019)", 
          'Hispanic People Killed by Police (1/1/2013-12/31/2019)', 
          'White People Killed by Police (1/1/2013-12/31/2019)']]
bigcities_killed = bigcities_killed.rename(columns = {"Black People Killed by Police (1/1/2013-12/31/2019)":'Black'})
bigcities_killed = bigcities_killed.rename(columns = {'Hispanic People Killed by Police (1/1/2013-12/31/2019)':'Hispanic'})
bigcities_killed = bigcities_killed.rename(columns = {'White People Killed by Police (1/1/2013-12/31/2019)':'White'})
bigcities_killed = bigcities_killed*1000000
bigcities_pop = bigcities [['Black', 'Hispanic', 'White']]
bigcities_ratio = (bigcities_killed/bigcities_pop)
bigcities_ratio = bigcities_ratio.transpose()
bigcities_ratio['Race'] = bigcities_ratio.index
bigcities_ratio = bigcities_ratio.rename(columns = {100:'Ratio'})

In [0]:
px.bar(bigcities_ratio, x = 'Race', y = 'Ratio', title = 'People Killed By Police by Race per 1 million people in Big Cities', opacity = 0.70, color = 'Race')

In [0]:
criminal_charges = policekillingsvictims[['Criminal Charges?']]
criminal_charges = criminal_charges.groupby('Criminal Charges?').size()
criminal_charges = criminal_charges.to_frame()
criminal_charges['Criminal Charges?'] = criminal_charges.index
criminal_charges = criminal_charges.rename(columns = {0:'Number of Charges'})
criminal_charges = pd.DataFrame(criminal_charges)
criminal_charges['Total'] = criminal_charges['Number of Charges'].sum()
criminal_charges = criminal_charges[['Number of Charges']].div(criminal_charges.Total, axis = 0)
criminal_charges['Criminal Charges?'] = criminal_charges.index
criminal_charges = criminal_charges.rename(columns = {0:'Number of Charges'})

In [0]:
criminal_charges.loc[criminal_charges['Number of Charges'] < 0.9, 'Criminal Charges?'] = 'Charged'
criminal_charges.loc[criminal_charges['Number of Charges'] > 0.9, 'Criminal Charges?'] = 'Not Charged'
fig = px.pie(criminal_charges, values = 'Number of Charges', names = 'Criminal Charges?', title = 'Charges against Police Brutality')
fig.show()

#Interpretations and conclusions

We began our analysis with the question: are the police actually arbiters of law and safety? We found out evidence pointing to the contrary. First we noticed that police are heavily predisposed to commit murders among black individuals, and secondly for hispanics. We also noticed that, for Republican states like Utah, Alaska, and Oklahoma, the rates of black killings were particularly high in contrast to the rates of all killings. This discovery is particularly resonant with the Black Lives Matter movement. One of their primary positions is that black lives are heavily targeted by police and are disproportionally killed by police. Our findings resonate with their claims. This obviously doesn't cover the whole story-- it would be of keen interest for us to examine the intricacies of each case, whether or not resisting was involved, and the severity of each of the individuals' actions and whether or not death was warranted. 


#Future directions
This study can go in a variety of directions. First of all, it would be really interesting to go deeper into the divide of Republican and Democrat and to see whether or not those states that are Republican are more likely to have police kill black individuals. Next, it would also be interesting to look at, again, the case to case details of each instance of a police killing. What factors are the most important in leading to a killing, outside of race? Finally, we would like to see changes in police killings over time. Are there certain events or instances during the years of 2013-2019 that contributed to a rise in police killings? Can we draw any links? These would be questions that can be answered as a continuation of our findings. 

